In [19]:
!nvcc --version
!pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118 --quiet

In [20]:
from google.colab import drive
drive.mount('/content/drive')

In [21]:
%cd '/content/'
!mkdir lpcc
%cd '/content/lpcc'

In [22]:
#replace with path to audio zip in your own google drive
audio_zip_path = '/content/drive/MyDrive/wavcaps stuff/audiocopy.zip'
%cd '/content/lpcc'

In [23]:
import os

final_dir = '/content/lpcc'
!mkdir test_set

get_file_id_from_google_drive_link = lambda file_link:file_link.split('/d/')[-1].split('/view')[0]
get_wget_command = lambda file_id,save_name: f'wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id={file_id}" -O- | sed -rn "s/.*confirm=([0-9A-Za-z_]+).*/\1\n/p")&id={file_id}" -O {save_name} && rm -rf ~/cookies.txt'

download_files_google_drive_link_list = [
    ('song_tags.csv','https://drive.google.com/file/d/1YYBxMD45hGGEo92m1iEo1KsbGU2CXcKp/view?usp=sharing'),
    ('split.json', 'https://drive.google.com/file/d/1eO2mmUz3WAlr1DEwcswY8mlB4DtBSqJc/view?usp=sharing')
]
for google_drive_link in download_files_google_drive_link_list:
  print(f'downloading: {google_drive_link[0]}')
  os.system(get_wget_command(get_file_id_from_google_drive_link(google_drive_link[1]),f'{final_dir}/{google_drive_link[0]}'))



In [8]:
import torch
import torchaudio

In [24]:
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [25]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
!pip -q install datasets

In [29]:
from datasets import load_dataset
pseudo_captions_dataset = load_dataset("seungheondoh/LP-MusicCaps-MSD")


In [30]:
pcd_test = pseudo_captions_dataset['test']
print(len(pcd_test))
print(pcd_test[0])

def get_caption_from_track_id(search_value_track_ID):
  matching_entry = next((entry for entry in pcd_test if entry["track_id"] == search_value_track_ID), None)
  if matching_entry:
      return matching_entry["caption_writing"]
  else:
      return ""


In [15]:
tracklist_file_path = '/content/drive/MyDrive/secondIdea/tracklist.txt'
def write_filename_to_tracklist(fname):
  with open(tracklist_file_path, 'a') as file:
      # Write the string to a new line
      file.write(fname + '\n')

In [16]:
split_path = '/content/lpcc/split.json'
!mkdir '/content/drive/My Drive/secondIdea'
!mkdir '/content/drive/My Drive/secondIdea/spectrograms'
!mkdir '/content/drive/My Drive/secondIdea/audios_temp'
paired_data_csv_file_path = '/content/drive/My Drive/secondIdea/caption_pairs.csv'
with open(paired_data_csv_file_path, 'w', newline='') as csv_file:
    pass
tracklist_path = '/content/drive/My Drive/secondIdea/tracklist.txt'
with open(tracklist_path, 'w') as text_file:
    pass

spectrograms_dir = '/content/drive/My Drive/secondIdea/spectrograms'
audios_temp_dir = '/content/drive/My Drive/secondIdea/audios_temp'


In [31]:
import json
import zipfile
import gc
import csv
import torchaudio
import matplotlib.pyplot as plt


# use only test track
with open(split_path, 'r') as f:
  split = json.load(f)
  test_track = split['test_track'] + split['extra_track']
  print(test_track)

len_test_track = len(test_track)

with zipfile.ZipFile(audio_zip_path, 'r') as zip_ref:
  with open(paired_data_csv_file_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    # csv_writer.writerow(["track_id", "caption"])
    len_test_track = len(test_track)
    for i in range(len_test_track):
      # print(i)
      gc.collect()
      torch.cuda.empty_cache()
      audio_file_to_extract = 'audio/' + test_track[i] + '.m4a'
      if audio_file_to_extract in zip_ref.namelist():
        zip_ref.extract(audio_file_to_extract, path=audios_temp_dir)

        # get groundtruth caption and write to csv
        groundtruth_caption = get_caption_from_track_id(test_track[i])
        pair_data_to_write = [test_track[i], groundtruth_caption]
        csv_writer.writerow(pair_data_to_write)

        # generate spectrogram
        waveform, sample_rate = torchaudio.load(audios_temp_dir + '/' + audio_file_to_extract)
        specgram = torchaudio.transforms.Spectrogram()(waveform)
        specgram_db = torchaudio.transforms.AmplitudeToDB()(specgram)
        specgram_np = specgram_db.numpy()[0]
        output_file_path = os.path.join(spectrograms_dir, test_track[i] + '.jpg')
        plt.imshow(specgram_np, cmap='viridis', aspect='auto', origin='lower')
        plt.axis('off')  # Turn off axis labels
        plt.savefig(output_file_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        # delete audio file
        if os.path.exists(audios_temp_dir + '/' + audio_file_to_extract):
          # Delete the file
          os.remove(audios_temp_dir + '/' + audio_file_to_extract)

      else:
        print(f"{audio_file_to_extract} does not exist in the archive.")

zip_ref.close()